In [1]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
import random

In [2]:
# Load the intents JSON file
with open('IPC_intents.json') as file:
    data = json.load(file)

In [3]:
# Extract patterns and corresponding tags
patterns = []
tags = []
tag_responses = {}

In [4]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])
    tag_responses[intent['tag']] = intent['responses']

In [5]:
# Vectorize the text data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(patterns).toarray()

In [6]:
# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(tags)

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Build a neural network model
model = Sequential()
model.add(Dense(128, input_shape=(X.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Train the model
model.fit(X_train, y_train, epochs=500, batch_size=8, verbose=1, validation_data=(X_test, y_test))

Epoch 1/500
533/533 [==============================] - 6s 10ms/step - loss: 1.9540 - accuracy: 0.5052 - val_loss: 1.4549 - val_accuracy: 0.7439
Epoch 2/500
533/533 [==============================] - 4s 8ms/step - loss: 1.9812 - accuracy: 0.5045 - val_loss: 1.4587 - val_accuracy: 0.7439
Epoch 3/500
533/533 [==============================] - 3s 7ms/step - loss: 1.9511 - accuracy: 0.5009 - val_loss: 1.4480 - val_accuracy: 0.7458
Epoch 4/500
533/533 [==============================] - 3s 6ms/step - loss: 2.0245 - accuracy: 0.4967 - val_loss: 1.4505 - val_accuracy: 0.7430
Epoch 5/500
533/533 [==============================] - 4s 7ms/step - loss: 1.9850 - accuracy: 0.4953 - val_loss: 1.4530 - val_accuracy: 0.7430
Epoch 6/500
533/533 [==============================] - 4s 8ms/step - loss: 1.9467 - accuracy: 0.5167 - val_loss: 1.4443 - val_accuracy: 0.7477
Epoch 7/500
533/533 [==============================] - 4s 7ms/step - loss: 1.9414 - accuracy: 0.5171 - val_loss: 1.4416 - val_accuracy: 0.743

In [13]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

34/34 [==============================] - 0s 3ms/step - loss: 1.5273 - accuracy: 0.7495
Accuracy: 74.95%


In [14]:
import pickle

# Save the vectorizer and label encoder
with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

# Save the responses
with open('tag_responses.pkl', 'wb') as file:
    pickle.dump(tag_responses, file)
    
# Save the model
model.save('chatbot_model.h5')

e:\Softwares\anaconda3\envs\chat-app\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
